In [1]:
!pip install tqdm -qq
from tqdm import tqdm

In [23]:
import json
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the NLLB model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-3.3B")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-3.3B").to('cuda')


# tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 23.64 GiB total capacity; 18.33 GiB already allocated; 60.25 MiB free; 18.37 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [6]:
def translate_entry(entry, target_language_code):
    # Function to translate individual entries
    if isinstance(entry, dict):
        entry_copy = entry.copy()
        

        for key, value in entry_copy.items():
            if key in ["Remark", "Suggestion", "Literal"]:
                # Tokenize the English text
                tokens = tokenizer(value, return_tensors="pt").input_ids[0].tolist()
                # If the text is longer than 512 tokens, split it into chunks
                chunks = [tokens[i:i + 512] for i in range(0, len(tokens), 512)]
                target_translation = ""
                for chunk in chunks:
                    # Translate each chunk to the target language
                    inputs = tokenizer(' '.join(chunk), return_tensors="pt").to(model.device)
                    translated_tokens = model.generate(
                        **inputs, 
                        forced_bos_token_id=tokenizer.lang_code_to_id[f"{target_language_code}"], 
                        max_length=512
                    )
                    target_translation += tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
                # Create the new key for the translated text
                new_key = f"{key}_{target_language_code}"
                entry[new_key] = target_translation  # Store new translation in original dictionary
            elif isinstance(value, dict):
                # Recurse into nested dictionaries
                translate_entry(value, target_language_code)
            elif isinstance(value, list):
                # Recurse into lists, item by item
                for item in value:
                    translate_entry(item, target_language_code)

def translate_to_target_language(target_language_code):
    # Load the JSON file
    with open("HOTTP.json", "r", encoding="utf-8") as file:
        data = json.load(file)
    # Get the HOTTP_Entry array
    entries = data['HOTTP_Entries']['HOTTP_Entry']
    # Process each entry in the HOTTP_Entry array
    for entry in tqdm(entries):  # Using tqdm to show progress
        translate_entry(entry, target_language_code)
    # Save the updated JSON data to a new file
    with open(f"HOTTP_translated_{target_language_code}.json", "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=2)

# Translate to a specific target language (e.g., 'spa_Latn')
translate_to_target_language('spa_Latn')


  0%|          | 0/4515 [00:00<?, ?it/s]


TypeError: sequence item 0: expected str instance, int found

In [ ]:
language_codes = ['ace_Arab', 'ace_Latn', 'acm_Arab', 'acq_Arab', 'aeb_Arab', 'afr_Latn', 'ajp_Arab', 'aka_Latn', 'amh_Ethi', 'apc_Arab', 'arb_Arab', 'arb_Latn', 'ars_Arab', 'ary_Arab', 'arz_Arab', 'asm_Beng', 'ast_Latn', 'awa_Deva', 'ayr_Latn', 'azb_Arab', 'azj_Latn', 'bak_Cyrl', 'bam_Latn', 'ban_Latn', 'bel_Cyrl', 'bem_Latn', 'ben_Beng', 'bho_Deva', 'bjn_Arab', 'bjn_Latn', 'bod_Tibt', 'bos_Latn', 'bug_Latn', 'bul_Cyrl', 'cat_Latn', 'ceb_Latn', 'ces_Latn', 'cjk_Latn', 'ckb_Arab', 'crh_Latn', 'cym_Latn', 'dan_Latn', 'deu_Latn', 'dik_Latn', 'dyu_Latn', 'dzo_Tibt', 'ell_Grek', 'eng_Latn', 'epo_Latn', 'est_Latn', 'eus_Latn', 'ewe_Latn', 'fao_Latn', 'fij_Latn', 'fin_Latn', 'fon_Latn', 'fra_Latn', 'fur_Latn', 'fuv_Latn', 'gla_Latn', 'gle_Latn', 'glg_Latn', 'grn_Latn', 'guj_Gujr', 'hat_Latn', 'hau_Latn', 'heb_Hebr', 'hin_Deva', 'hne_Deva', 'hrv_Latn', 'hun_Latn', 'hye_Armn', 'ibo_Latn', 'ilo_Latn', 'ind_Latn', 'isl_Latn', 'ita_Latn', 'jav_Latn', 'jpn_Jpan', 'kab_Latn', 'kac_Latn', 'kam_Latn', 'kan_Knda', 'kas_Arab', 'kas_Deva', 'kat_Geor', 'knc_Arab', 'knc_Latn', 'kaz_Cyrl', 'kbp_Latn', 'kea_Latn', 'khm_Khmr', 'kik_Latn', 'kin_Latn', 'kir_Cyrl', 'kmb_Latn', 'kmr_Latn', 'kon_Latn', 'kor_Hang', 'lao_Laoo', 'lij_Latn', 'lim_Latn', 'lin_Latn', 'lit_Latn', 'lmo_Latn', 'ltg_Latn', 'ltz_Latn', 'lua_Latn', 'lug_Latn', 'luo_Latn', 'lus_Latn', 'lvs_Latn', 'mag_Deva', 'mai_Deva', 'mal_Mlym', 'mar_Deva', 'min_Arab', 'min_Latn', 'mkd_Cyrl', 'plt_Latn', 'mlt_Latn', 'mni_Beng', 'khk_Cyrl', 'mos_Latn', 'mri_Latn', 'mya_Mymr', 'nld_Latn', 'nno_Latn', 'nob_Latn', 'npi_Deva', 'nso_Latn', 'nus_Latn', 'nya_Latn', 'oci_Latn', 'gaz_Latn', 'ory_Orya', 'pag_Latn', 'pan_Guru', 'pap_Latn', 'pes_Arab', 'pol_Latn', 'por_Latn', 'prs_Arab', 'pbt_Arab', 'quy_Latn', 'ron_Latn', 'run_Latn', 'rus_Cyrl', 'sag_Latn', 'san_Deva', 'sat_Olck', 'scn_Latn', 'shn_Mymr', 'sin_Sinh', 'slk_Latn', 'slv_Latn', 'smo_Latn', 'sna_Latn', 'snd_Arab', 'som_Latn', 'sot_Latn', 'spa_Latn', 'als_Latn', 'srd_Latn', 'srp_Cyrl', 'ssw_Latn', 'sun_Latn', 'swe_Latn', 'swh_Latn', 'szl_Latn', 'tam_Taml', 'tat_Cyrl', 'tel_Telu', 'tgk_Cyrl', 'tgl_Latn', 'tha_Thai', 'tir_Ethi', 'taq_Latn', 'taq_Tfng', 'tpi_Latn', 'tsn_Latn', 'tso_Latn', 'tuk_Latn', 'tum_Latn', 'tur_Latn', 'twi_Latn', 'tzm_Tfng', 'uig_Arab', 'ukr_Cyrl', 'umb_Latn', 'urd_Arab', 'uzn_Latn', 'vec_Latn', 'vie_Latn', 'war_Latn', 'wol_Latn', 'xho_Latn', 'ydd_Hebr', 'yor_Latn', 'yue_Hant', 'zho_Hans', 'zho_Hant', 'zsm_Latn', 'zul_Latn']

# Translate to all languages
# Skip English, French, and Spanish, since they are already in the HOTTP.json file or already translated
for language_code in language_codes:
    if language_code != 'eng_Latn' and language_code != 'spa_Latn' and language_code != 'fra_Latn':
        translate_to_target_language(language_code)

## Localize factors and ratings

In [19]:
language_codes = ['ace_Arab', 'ace_Latn', 'acm_Arab', 'acq_Arab', 'aeb_Arab', 'afr_Latn', 'ajp_Arab', 'aka_Latn', 'amh_Ethi', 'apc_Arab', 'arb_Arab', 'arb_Latn', 'ars_Arab', 'ary_Arab', 'arz_Arab', 'asm_Beng', 'ast_Latn', 'awa_Deva', 'ayr_Latn', 'azb_Arab', 'azj_Latn', 'bak_Cyrl', 'bam_Latn', 'ban_Latn', 'bel_Cyrl', 'bem_Latn', 'ben_Beng', 'bho_Deva', 'bjn_Arab', 'bjn_Latn', 'bod_Tibt', 'bos_Latn', 'bug_Latn', 'bul_Cyrl', 'cat_Latn', 'ceb_Latn', 'ces_Latn', 'cjk_Latn', 'ckb_Arab', 'crh_Latn', 'cym_Latn', 'dan_Latn', 'deu_Latn', 'dik_Latn', 'dyu_Latn', 'dzo_Tibt', 'ell_Grek', 'eng_Latn', 'epo_Latn', 'est_Latn', 'eus_Latn', 'ewe_Latn', 'fao_Latn', 'fij_Latn', 'fin_Latn', 'fon_Latn', 'fra_Latn', 'fur_Latn', 'fuv_Latn', 'gla_Latn', 'gle_Latn', 'glg_Latn', 'grn_Latn', 'guj_Gujr', 'hat_Latn', 'hau_Latn', 'heb_Hebr', 'hin_Deva', 'hne_Deva', 'hrv_Latn', 'hun_Latn', 'hye_Armn', 'ibo_Latn', 'ilo_Latn', 'ind_Latn', 'isl_Latn', 'ita_Latn', 'jav_Latn', 'jpn_Jpan', 'kab_Latn', 'kac_Latn', 'kam_Latn', 'kan_Knda', 'kas_Arab', 'kas_Deva', 'kat_Geor', 'knc_Arab', 'knc_Latn', 'kaz_Cyrl', 'kbp_Latn', 'kea_Latn', 'khm_Khmr', 'kik_Latn', 'kin_Latn', 'kir_Cyrl', 'kmb_Latn', 'kmr_Latn', 'kon_Latn', 'kor_Hang', 'lao_Laoo', 'lij_Latn', 'lim_Latn', 'lin_Latn', 'lit_Latn', 'lmo_Latn', 'ltg_Latn', 'ltz_Latn', 'lua_Latn', 'lug_Latn', 'luo_Latn', 'lus_Latn', 'lvs_Latn', 'mag_Deva', 'mai_Deva', 'mal_Mlym', 'mar_Deva', 'min_Arab', 'min_Latn', 'mkd_Cyrl', 'plt_Latn', 'mlt_Latn', 'mni_Beng', 'khk_Cyrl', 'mos_Latn', 'mri_Latn', 'mya_Mymr', 'nld_Latn', 'nno_Latn', 'nob_Latn', 'npi_Deva', 'nso_Latn', 'nus_Latn', 'nya_Latn', 'oci_Latn', 'gaz_Latn', 'ory_Orya', 'pag_Latn', 'pan_Guru', 'pap_Latn', 'pes_Arab', 'pol_Latn', 'por_Latn', 'prs_Arab', 'pbt_Arab', 'quy_Latn', 'ron_Latn', 'run_Latn', 'rus_Cyrl', 'sag_Latn', 'san_Deva', 'sat_Olck', 'scn_Latn', 'shn_Mymr', 'sin_Sinh', 'slk_Latn', 'slv_Latn', 'smo_Latn', 'sna_Latn', 'snd_Arab', 'som_Latn', 'sot_Latn', 'spa_Latn', 'als_Latn', 'srd_Latn', 'srp_Cyrl', 'ssw_Latn', 'sun_Latn', 'swe_Latn', 'swh_Latn', 'szl_Latn', 'tam_Taml', 'tat_Cyrl', 'tel_Telu', 'tgk_Cyrl', 'tgl_Latn', 'tha_Thai', 'tir_Ethi', 'taq_Latn', 'taq_Tfng', 'tpi_Latn', 'tsn_Latn', 'tso_Latn', 'tuk_Latn', 'tum_Latn', 'tur_Latn', 'twi_Latn', 'tzm_Tfng', 'uig_Arab', 'ukr_Cyrl', 'umb_Latn', 'urd_Arab', 'uzn_Latn', 'vec_Latn', 'vie_Latn', 'war_Latn', 'wol_Latn', 'xho_Latn', 'ydd_Hebr', 'yor_Latn', 'yue_Hant', 'zho_Hans', 'zho_Hant', 'zsm_Latn', 'zul_Latn']


factors_file = 'HOTTP-FACTORS-EN.HTML'
ratings_file = 'HOTTP-RATINGS-EN.HTML'

# In each file, we want to translate every 'content' field, which could be a string or an array of strings that should be joined.

def translate_entry(entry, target_language_code):
    # Function to translate individual entries
    if isinstance(entry, dict):
        entry_copy = entry.copy()
        

        for key, value in entry_copy.items():
            if key in ["content"]:
                # Tokenize the English text
                tokens = tokenizer(value, return_tensors="pt").input_ids[0].tolist()
                # If the text is longer than 512 tokens, split it into chunks
                chunks = [tokens[i:i + 512] for i in range(0, len(tokens), 512)]
                target_translation = ""
                for chunk in chunks:
                    # Translate each chunk to the target language
                    inputs = tokenizer(' '.join(chunk), return_tensors="pt").to(model.device)
                    translated_tokens = model.generate(
                        **inputs, 
                        forced_bos_token_id=tokenizer.lang_code_to_id[f"{target_language_code}"], 
                        max_length=512
                    )
                    target_translation += tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
                # Create the new key for the translated text
                new_key = f"{key}_{target_language_code}"
                entry[new_key] = target_translation  # Store new translation in original dictionary
            elif isinstance(value, dict):
                # Recurse into nested dictionaries
                translate_entry(value, target_language_code)
            elif isinstance(value, list):
                # Recurse into lists, item by item
                for item in value:
                    translate_entry(item, target_language_code)

def translate_to_target_language(target_language_code, input_file):
    # Load the JSON file
    with open(input_file, "r", encoding="utf-8") as file:
        data = json.load(file)
    # Get the HOTTP_Entry array
    entries = data['html']['body']
    # Process each entry in the HOTTP_Entry array
    for entry in tqdm(entries):  # Using tqdm to show progress
        translate_entry(entry, target_language_code)
    # Save the updated JSON data to a new file
    with open(f"{input_file}_{target_language_code}.json", "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=2)

# Translate to a specific target language (e.g., 'spa_Latn')
translate_to_target_language('spa_Latn', factors_file)
translate_to_target_language('spa_Latn', ratings_file)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [27]:

from bs4 import BeautifulSoup

language_codes = ['ace_Arab', 'ace_Latn', 'acm_Arab', 'acq_Arab', 'aeb_Arab', 'afr_Latn', 'ajp_Arab', 'aka_Latn', 'amh_Ethi', 'apc_Arab', 'arb_Arab', 'arb_Latn', 'ars_Arab', 'ary_Arab', 'arz_Arab', 'asm_Beng', 'ast_Latn', 'awa_Deva', 'ayr_Latn', 'azb_Arab', 'azj_Latn', 'bak_Cyrl', 'bam_Latn', 'ban_Latn', 'bel_Cyrl', 'bem_Latn', 'ben_Beng', 'bho_Deva', 'bjn_Arab', 'bjn_Latn', 'bod_Tibt', 'bos_Latn', 'bug_Latn', 'bul_Cyrl', 'cat_Latn', 'ceb_Latn', 'ces_Latn', 'cjk_Latn', 'ckb_Arab', 'crh_Latn', 'cym_Latn', 'dan_Latn', 'deu_Latn', 'dik_Latn', 'dyu_Latn', 'dzo_Tibt', 'ell_Grek', 'eng_Latn', 'epo_Latn', 'est_Latn', 'eus_Latn', 'ewe_Latn', 'fao_Latn', 'fij_Latn', 'fin_Latn', 'fon_Latn', 'fra_Latn', 'fur_Latn', 'fuv_Latn', 'gla_Latn', 'gle_Latn', 'glg_Latn', 'grn_Latn', 'guj_Gujr', 'hat_Latn', 'hau_Latn', 'heb_Hebr', 'hin_Deva', 'hne_Deva', 'hrv_Latn', 'hun_Latn', 'hye_Armn', 'ibo_Latn', 'ilo_Latn', 'ind_Latn', 'isl_Latn', 'ita_Latn', 'jav_Latn', 'jpn_Jpan', 'kab_Latn', 'kac_Latn', 'kam_Latn', 'kan_Knda', 'kas_Arab', 'kas_Deva', 'kat_Geor', 'knc_Arab', 'knc_Latn', 'kaz_Cyrl', 'kbp_Latn', 'kea_Latn', 'khm_Khmr', 'kik_Latn', 'kin_Latn', 'kir_Cyrl', 'kmb_Latn', 'kmr_Latn', 'kon_Latn', 'kor_Hang', 'lao_Laoo', 'lij_Latn', 'lim_Latn', 'lin_Latn', 'lit_Latn', 'lmo_Latn', 'ltg_Latn', 'ltz_Latn', 'lua_Latn', 'lug_Latn', 'luo_Latn', 'lus_Latn', 'lvs_Latn', 'mag_Deva', 'mai_Deva', 'mal_Mlym', 'mar_Deva', 'min_Arab', 'min_Latn', 'mkd_Cyrl', 'plt_Latn', 'mlt_Latn', 'mni_Beng', 'khk_Cyrl', 'mos_Latn', 'mri_Latn', 'mya_Mymr', 'nld_Latn', 'nno_Latn', 'nob_Latn', 'npi_Deva', 'nso_Latn', 'nus_Latn', 'nya_Latn', 'oci_Latn', 'gaz_Latn', 'ory_Orya', 'pag_Latn', 'pan_Guru', 'pap_Latn', 'pes_Arab', 'pol_Latn', 'por_Latn', 'prs_Arab', 'pbt_Arab', 'quy_Latn', 'ron_Latn', 'run_Latn', 'rus_Cyrl', 'sag_Latn', 'san_Deva', 'sat_Olck', 'scn_Latn', 'shn_Mymr', 'sin_Sinh', 'slk_Latn', 'slv_Latn', 'smo_Latn', 'sna_Latn', 'snd_Arab', 'som_Latn', 'sot_Latn', 'spa_Latn', 'als_Latn', 'srd_Latn', 'srp_Cyrl', 'ssw_Latn', 'sun_Latn', 'swe_Latn', 'swh_Latn', 'szl_Latn', 'tam_Taml', 'tat_Cyrl', 'tel_Telu', 'tgk_Cyrl', 'tgl_Latn', 'tha_Thai', 'tir_Ethi', 'taq_Latn', 'taq_Tfng', 'tpi_Latn', 'tsn_Latn', 'tso_Latn', 'tuk_Latn', 'tum_Latn', 'tur_Latn', 'twi_Latn', 'tzm_Tfng', 'uig_Arab', 'ukr_Cyrl', 'umb_Latn', 'urd_Arab', 'uzn_Latn', 'vec_Latn', 'vie_Latn', 'war_Latn', 'wol_Latn', 'xho_Latn', 'ydd_Hebr', 'yor_Latn', 'yue_Hant', 'zho_Hans', 'zho_Hant', 'zsm_Latn', 'zul_Latn']


factors_file = 'HOTTP-FACTORS-EN.HTML'
ratings_file = 'HOTTP-RATINGS-EN.HTML'

def translate_entry(entry, target_language_code):
    # Function to translate individual entries
    if isinstance(entry, str):
        inputs = tokenizer(entry, return_tensors="pt", padding=True).to(model.device)
        translated_tokens = model.generate(
            **inputs, 
            forced_bos_token_id=tokenizer.lang_code_to_id[f"{target_language_code}"], 
            max_length=512
        )
        return tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

from bs4 import NavigableString
def translate_to_target_language(target_language_code, input_file):
    # Load the HTML file
    with open(input_file, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, 'html.parser')
    # Get the body of the HTML
    body = soup.body
        

    # Check if body is not None
    if body is not None:
        for string in tqdm(list(body.stripped_strings)):  # Use list() to avoid issues during iteration and modification
            translated_text = translate_entry(string, target_language_code)
            # Find the exact text node in the soup and replace its content
            for text_node in body.find_all(text=True):
                if text_node == string and isinstance(text_node, NavigableString):
                    text_node.replace_with(translated_text)
                    break  # Break out of the loop once we've found and replaced the node

        # Update the lang attribute
        soup.body['lang'] = target_language_code

    # Save the updated HTML data to a new file
    with open(f"{input_file}_{target_language_code}.html", "w", encoding="utf-8") as file:
        file.write(str(soup))


# Translate to a specific target language (e.g., 'spa_Latn')
translate_to_target_language('spa_Latn', factors_file)
translate_to_target_language('spa_Latn', ratings_file)

  0%|          | 0/43 [00:00<?, ?it/s]/tmp/ipykernel_1237730/3971437054.py:34: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for text_node in body.find_all(text=True):
100%|██████████| 26/26 [00:17<00:00,  1.48it/s]
